# Multiple dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
f(x) = x.^2

f (generic function with 1 method)

and then Julia will determine on its own which input argument types make sense and which do not:

In [2]:
f(10)

100

In [3]:
f([1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `foo` that only takes strings as inputs.

In [4]:
foo(x::String, y::String) = println("My inputs x and y are both strings!")

foo (generic function with 1 method)

We see here that in order to restrict the type of `x` and `y` to `String`s, we just follow the input argument name by a double colon and the keyword `String`.

Now we'll see that `foo` works on `String`s and doesn't work on other input argument types.

In [5]:
foo("hello", "hi!")

My inputs x and y are both strings!


In [6]:
foo(3, 4)

MethodError: MethodError: no method matching foo(::Int64, ::Int64)

To get `foo` to work on integer (`Int`) inputs, let's tack `::Int` onto our input arguments when we declare `foo`.

In [7]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [8]:
foo(3, 4)

My inputs x and y are both integers!


Now `foo` works on integers! But look, `foo` also still works when `x` and `y` are strings!

In [9]:
foo("hello", "hi!")

My inputs x and y are both strings!


This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(x::String, y::String)
```
Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [10]:
methods(foo)

# 2 methods for generic function "foo" from Main:
 [1] foo(x::Int64, y::Int64)
     @ c:\Data\Code\Introduction-to-Julia\8 - Multiple dispatch.ipynb:1
 [2] foo(x::String, y::String)
     @ c:\Data\Code\Introduction-to-Julia\8 - Multiple dispatch.ipynb:1

Aside: how many methods do you think there are for addition?

In [11]:
methods(+)

# 189 methods for generic function "+" from Base:
   [1] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\periods.jl:334
   [2] +(x::Dates.CompoundPeriod, y::Dates.TimeType)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\periods.jl:362
   [3] +(x::Dates.CompoundPeriod, y::Dates.Period)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\periods.jl:332
   [4] +(x::Bool, z::Complex{Bool})
     @ complex.jl:305
   [5] +(x::Bool, y::Bool)
     @ bool.jl:166
   [6] +(x::Bool)
     @ bool.jl:163
   [7] +(x::Bool, z::Complex)
     @ complex.jl:312
   [8] +(x::Real, z::Complex{Bool})
     @ complex.jl:319
   [9] +(x::Bool, y::T) where T<:AbstractFloat
     @ bool.jl:173
  [10] +(x::Dates.DateTime, y::Dates.Quarter)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:77
  [11] +(dt::Dates.DateTime, z::Dates.Month)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:49
  [12] +(dt::Dates.DateTime, y::Dates.Year)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:25
  [13] +(x::Dates.DateTime, y::Dates.Period)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:83
  [14] +(a::Pkg.Resolve.FieldValue, b::Pkg.Resolve.FieldValue)
     @ Pkg.Resolve C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Pkg\src\Resolve\fieldvalues.jl:43
  [15] +(z::Complex{Bool}, x::Bool)
     @ complex.jl:306
  [16] +(z::Complex{Bool}, x::Real)
     @ complex.jl:320
  [17] +(z::Complex, x::Bool)
     @ complex.jl:313
  [18] +(::Missing, ::Missing)
     @ missing.jl:122
  [19] +(::Missing)
     @ missing.jl:101
  [20] +(x::Missing, y::Dates.AbstractTime)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:91
  [21] +(::Missing, ::Number)
     @ missing.jl:123
  [22] +(level::Base.CoreLogging.LogLevel, inc::Integer)
     @ Base.CoreLogging logging.jl:131
  [23] +(x::Dates.Date, y::Dates.Day)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:81
  [24] +(x::Dates.Date, y::Dates.Week)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:79
  [25] +(x::Dates.Date, y::Dates.Quarter)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:75
  [26] +(dt::Dates.Date, z::Dates.Month)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:56
  [27] +(dt::Dates.Date, y::Dates.Year)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:29
  [28] +(dt::Dates.Date, t::Dates.Time)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:21
  [29] +(t::Dates.Time, dt::Dates.Date)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:22
  [30] +(x::Dates.Time, y::Dates.TimePeriod)
     @ Dates C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Dates\src\arithmetic.jl:85
  [31] +(x::Rational{BigInt}, y::Rational{BigInt})
     @ Base.GMP.MPQ gmp.jl:1061
  [32] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight)
     @ Pkg.Resolve C:\Users\aaath\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\Pkg\src\Resolve\versionweights.jl:22
  [3

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [12]:
@which foo(3, 4)

foo(x::Int64, y::Int64)
     @ Main c:\Data\Code\Introduction-to-Julia\8 - Multiple dispatch.ipynb:1

Let's see what happens when we use `@which` with the addition operator!

In [13]:
@which 3.0 + 3.0

+(x::T, y::T) where T<:Union{Float16, Float32, Float64}
     @ Base float.jl:409

And we can continue to add other methods to our generic function `foo`. Let's add one that takes the ***abstract type*** `Number`, which includes subtypes such as `Int`, `Float64`, and other objects you would think of as numbers:

In [14]:
foo(x::Number, y::Number) = println("My inputs x and y are both numbers!")

foo (generic function with 3 methods)

This method for `foo` will work on, for example, floating point numbers:

In [15]:
foo(3.0, 4.0)

My inputs x and y are both numbers!


We can also add a fallback, duck-typed method for `foo` that takes inputs of any type:

In [16]:
foo(x, y) = println("I accept inputs of any type!")

foo (generic function with 4 methods)

Given the methods we've already written for `foo` so far, this method will be called whenever we pass non-numbers to `foo`:

In [17]:
v = rand(3)
foo(v, v)

I accept inputs of any type!


### Exercises

#### 9.1

Extend the function `foo`, adding a method that takes only one input argument, which is of type `Bool`, and returns "foo with one boolean!"

In [27]:
foo(x :: Bool) = "foo with one boolean!"

foo (generic function with 5 methods)

#### 9.2

Check that the method being dispatched when you execute 
```julia
foo(true)
```
is the one you wrote.

In [28]:
foo(true)

"foo with one boolean!"

In [29]:
@assert foo(true) == "foo with one boolean!"